In [1]:
import os
import xarray
from datetime import datetime, timedelta
import numpy

In [2]:
ERA5_ALBEDO_FOLDER = '/disk2/Data/ERA5_Albedo'

In [3]:
UTC_OFFSET = 9 # hour
time_internal = 1  # hour

tky_site_loc = (36.146, 137.423) # lat, lon
crk_site_loc = (38.201, 127.251) # lat, lon
gck_site_loc = (37.748, 127.162) # lat, lon
gdk_site_loc = (37.749, 127.149) # lat, lon
site_names = ['TKY','CRK', 'GCK', 'GDK']
site_locs = [tky_site_loc, crk_site_loc, gck_site_loc, gdk_site_loc]

In [4]:
def read_site_era5_albedo_month(era5_albedo_grib, lat_site, lon_site):
    albedo_ds = xarray.load_dataset(era5_albedo_grib, engine="cfgrib")
    albedo_site_ds = albedo_ds.interp(longitude=[lon_site], latitude=[lat_site], method="nearest", kwargs={"fill_value": "extrapolate"}) # linear
    
    albedo_v = numpy.array(albedo_site_ds.to_array()[0,1:,:,0,0])
    albedo_v_1d = albedo_v.flatten()
    albedo_v_1d = numpy.insert(albedo_v_1d, 0, albedo_site_ds.to_array()[0,0,23,0,0])
    albedo_v_1d = numpy.delete(albedo_v_1d, -1)
    
    return albedo_v_1d

In [5]:
for current_site_idx in range(len(site_names)):
    current_site_loc = site_locs[current_site_idx]

    years = ['2017', '2018', '2019']
    months = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12']

    data_record = []
    for year_str in years:
        for month_str in months:
            year_month = year_str + '_' + month_str
            grib_filename = os.path.join(ERA5_ALBEDO_FOLDER, year_month + '.grib')
            if os.path.exists(grib_filename):
                utc_start_date = datetime.strptime(year_month + '-01T00:00:00Z', "%Y_%m-%dT%H:%M:%SZ")
                file_site_v_list = read_site_era5_albedo_month(grib_filename, current_site_loc[0], current_site_loc[1])
                current_date = utc_start_date
                for site_v in file_site_v_list:
                    local_current_date = current_date + timedelta(hours=UTC_OFFSET)
                    local_current_time = local_current_date.strftime("%Y-%m-%dT%H:%M:%SZ")
                    data_record.append([local_current_time, str(site_v)])
                    current_date = current_date + timedelta(hours=time_internal)
    albedo_site_data = numpy.array(data_record)
    
    numpy.savetxt(os.path.join(ERA5_ALBEDO_FOLDER, site_names[current_site_idx]+'_ERA5_Albedo'+'.csv'), albedo_site_data, delimiter=",", fmt='%s')

/home/beichen/anaconda3/envs/python36/lib/python3.6/site-packages/xarray/backends/plugins.py:61: RuntimeWarning: Engine 'cfgrib' loading failed:
module 'cfgrib.messages' has no attribute 'DEFAULT_INDEXPATH'
  warnings.warn(f"Engine {name!r} loading failed:\n{ex}", RuntimeWarning)
